For working with the model, we first import some packages and the modelclass:

In [1]:
#os.chdir('C:\\Users\\mpher\\Documents\\Uni\\Master\\02_Exchange\\01_Academics\\Energy Economics of the Green Transition\\0_Final Project\EnergyEconGroupWork')

In [2]:
import os
current_directory = os.getcwd()
print(current_directory)

/Users/paul/EnergyEcon/EnergyEconGroupWork


In [3]:
%run ./stdPackages.ipynb
from pyDbs import *

os.chdir(os.path.join(d['curr'], 'py'))

from mBasicPH_storage import *
os.chdir(d['curr'])

Next we load the database:

In [4]:
kwargs = {'variables': ['Fundamentals', 'LoadVariables', 'StorageVariables', 'GeneratorsVariables'],#'TransmissionLines'],
        'maps': ['LoadMaps','GeneratorsMaps','StorageMaps', 'MarketMaps','hMaps'],
        'variable2D': ['HourlyVariation']}

path_to_data = os.path.join(d['curr'], 'Data', 'mBasicPH_storage_Data.xlsx')

db = read.dbFromWB(path_to_data, kwargs)
readSets(db)

Convert variables to floats:

In [5]:
[db.__setitem__(var,db[var].astype(float).sort_index()) for var in db.getTypes('variable').keys()];

We need to read in heat areas manually:

In [6]:
db['g_H'] = db['g_E2g_H'].get_level_values('g_H')

We need to read in electricity areas manually:

In [7]:
db['g_E'] = db['g_E2g'].get_level_values('g_E')

Collect all areas in one:

In [8]:
db['g'] = db['g_E'].copy()

Finally, we will assume that linelosses and trade are zero such that the marginal willingness to pay and marginal generation costs of "transmission lines" does not have to be corrected. 

We then initialize and solve the model:

In [9]:
m = mSimple(db)
m.db.symbols.keys()

TypeError: Symbol lineCapacity not in database

In [ ]:
m()

In [ ]:
#df = pd.DataFrame(db['x'])
#df.to_excel(os.path.join(d['curr'], 'x.xlsx'), engine='openpyxl')

#db['x']

What to plot?

- CO2 Emissions
- Usage of Storage
- Marginal Economic value of storage
- seasonal storage?